imports

In [6]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel, SelectKBest, mutual_info_classif
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression #(setting multi_class=”multinomial”)
from sklearn.model_selection import train_test_split
import os

Read data

In [7]:
path = ""

def read_data(csvfile):
    df = pd.read_csv(csvfile)
    return df
#read data
trainFile = os.path.join(path, "train.csv")
train_df = read_data(trainFile)

In [8]:
#summary statistics of train data
summary = train_df.describe()
#count the number of unique levels for each feature as well as for Target 
nb_levels = pd.DataFrame()
cols = train_df.columns
for j in range(train_df.shape[1]):
    l = train_df[cols[j]].unique()
    nb_levels[cols[j]] = pd.Series(len(l))
    #distribution of levels for each feature
#    for feat in cols[1:]:
#        figName = feat+'.png'
#        f = plt.figure()
#        sns.countplot(x=feat, data= train_df,
#             order = train_df[feat].value_counts().index, palette="Greens_d")
#        plt.xticks(rotation=85, fontsize = 8)
#        f.savefig(os.path.join(path, "EDA", figName))

study feature correlation 

In [9]:
corr = train_df[cols[1:-1]].corr()
#seperate features and target
X = train_df[cols[1:-1]]
y = train_df[cols[-1]]
pca = PCA()
pca.fit(X)
exp_var_ratio = pca.explained_variance_ratio_

feature selection

In [10]:
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model1 = SelectFromModel(lsvc, prefit=True)
idx1 = model1.get_support(indices=True)
X_new = model1.transform(X)
X_new.shape
feat_selected1 = list(X.columns[idx1])

In [11]:
clf = ExtraTreesClassifier()
clf = clf.fit(X, y)
model2 = SelectFromModel(clf, prefit=True)
idx2 = model2.get_support(indices=True)
X_new = model2.transform(X)
X_new.shape   
feat_selected2 = list(X.columns[idx2])

In [12]:
model3 = SelectKBest(mutual_info_classif, k=40).fit(X,y)
idx3 = model3.get_support(indices=True)
X_new = model3.transform(X) 
X_new.shape
feat_selected3 = list(X.columns[idx3])

In [17]:
#spit data into train/test/validation sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size =0.2)

In [22]:
from sklearn.model_selection import StratifiedShuffleSplit

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=0)
for train_index, test_index in sss.split(X_train, y_train):
    X_train_, X_val_ = X_train[train_index], X_train[test_index]
    y_train_, y_val_ = y_train[train_index], y_train[test_index]

In [23]:
#try out some classification models
models = [KNeighborsClassifier(5), RandomForestClassifier(), LogisticRegression()]
accuracy = []
for clf in models:
    clf.fit(X_train, y_train)
    mean_acc = clf.score(X_test, y_test)
    accuracy.append(mean_acc)

In [24]:
print(accuracy)

[0.77731092436974791, 0.77359405300581774, 0.75468648998060761]
